*NOTE: sumstats still in hg37*

### **Setup:**

In [1]:
.libPaths(c(.libPaths(), "/exports/eddie3_apps_local/apps/SL7/R/4.3.0/lib64/R/library"))
library("dplyr", warn.conflicts = F)
library("tidyverse")
library("stringr", warn.conflicts = F)
library("data.table", warn.conflicts = F) 
library("vroom", warn.conflicts = F)
library("ggplot2", warn.conflicts = F)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.5
✔ ggplot2   3.5.1     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


### **Change working directory:**

In [2]:
wd = "/exports/cmvm/eddie/smgphs/groups/Quantgen/Users/vasilis/PHD/"
setwd(wd)
getwd()

[1] "/exports/cmvm/eddie/smgphs/groups/Quantgen/Users/vasilis/PHD"

### **Load data:**

In [3]:
nonprot <-
  list.files(path="ukb.delirium.gwas/non_prot_eur/output", pattern = "ukb_nonprot_eur_c.*_delirium.regenie", full.names=T) %>%
  map_df(~fread(.))

### **QC**:


In [4]:
nonprot_qc <- 
    nonprot %>% 
    # make pvalue 
    mutate(P = 10^(-1*LOG10P)) %>%
    # make maf
    mutate(MAF = ifelse(A1FREQ > 0.5, 1 - A1FREQ, A1FREQ)) %>%
    mutate(MAF_CASES = ifelse(A1FREQ_CASES > 0.5, 1 - A1FREQ_CASES, A1FREQ_CASES)) %>%
    mutate(MAF_CONTROLS = ifelse(A1FREQ_CONTROLS > 0.5, 1 - A1FREQ_CONTROLS, A1FREQ_CONTROLS)) %>%
    # make mac 
    mutate(MAC = 2*N*MAF) %>% 
    mutate(MAC_CASES = 2*N_CASES*MAF_CASES) %>%
    mutate(MAC_CONTROLS = 2*N_CONTROLS*MAF_CONTROLS) %>% 
    # filter MAC>=5 on cases and controls
    filter(MAC_CASES >= 5 & MAC_CONTROLS >= 5) %>%
    # filter INFO >= 0.5 
    filter(INFO >= 0.5) %>%
    # select
    select(CHROM, GENPOS, ID, ALLELE0, ALLELE1, A1FREQ,A1FREQ_CASES, A1FREQ_CONTROLS, MAF, MAC, MAF_CASES, MAC_CASES, MAF_CONTROLS, MAC_CONTROLS, BETA, SE, INFO, P)
nonprot_qc %>% head

CHROM,GENPOS,ID,ALLELE0,ALLELE1,A1FREQ,A1FREQ_CASES,A1FREQ_CONTROLS,MAF,MAC,MAF_CASES,MAC_CASES,MAF_CONTROLS,MAC_CONTROLS,BETA,SE,INFO,P
<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,13483,rs554760071,G,C,0.005014410,0.004875420,0.005017010,0.005014410,3619.4112,0.004875420,64.843086,0.005017010,3554.5616,-0.0382893,0.1777700,0.500866,0.8294662
1,14975,rs11803341,C,T,0.000614720,0.000505086,0.000616778,0.000614720,443.7061,0.000505086,6.717644,0.000616778,436.9884,-0.2480200,0.4715280,0.608614,0.5988941
1,17641,rs578081284,G,A,0.000825771,0.000936754,0.000823688,0.000825771,596.0432,0.000936754,12.458828,0.000823688,583.5846,0.2985120,0.3499940,0.858936,0.3937105
1,18849,rs533090414,C,G,0.975644000,0.975293000,0.975650000,0.024356000,17580.2095,0.024707000,328.603100,0.024350000,17252.0237,-0.0330117,0.0801674,0.511276,0.6804982
1,30524,rs534702355,G,A,0.000600355,0.000589415,0.000600560,0.000600355,433.3374,0.000589415,7.839220,0.000600560,425.4980,-0.0837607,0.4473000,0.618697,0.8514579
1,49318,rs536836601,A,G,0.001395820,0.001381390,0.001396090,0.001395820,1007.5057,0.001381390,18.372487,0.001396090,989.1326,-0.0304761,0.3207220,0.541904,0.9242963


In [5]:
cat(paste0("Variants in nonprot UKB EUR GWAS: ", prettyNum(nrow(nonprot_qc),big.mark = ",")))


Variants in nonprot UKB EUR GWAS: 21,976,921

### Write output:


In [6]:
write.table(nonprot_qc, file="ukb.delirium.gwas/non_prot_eur/output/ukb_nonprot_eur_fgQC_all.regenie", quote=F, col.names=T, row.names=F)